In [49]:
from bs4 import BeautifulSoup
from lxml import etree
import requests
import re
import pandas as pd

In [52]:
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor

# Define uma função para obter os links de uma página
def get_links(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
    }

    response = requests.get(url, headers=headers)
    html_content = response.text
    soup = BeautifulSoup(html_content, 'html.parser')

    matching_elements = soup.find_all('a', class_='box-texto-relacionado')
    return [element.attrs['href'] for element in matching_elements]

# Define a lista de URLs a serem requisitadas

base_url = 'https://www.cepea.esalq.usp.br/br/diarias-de-mercado/arroz-cepea-demanda-aquecida-mantem-indicador-firme.aspx?pagina='
urls = []
for i in range(815):
    url = base_url + str(i)
    urls.append(url)

# Executa as requisições em paralelo e armazena os links em um único vetor
links = []
with ThreadPoolExecutor() as executor:
    for result in executor.map(get_links, urls):
        links += result

print(len(links))

15


In [57]:
def obter_noticias(url: str):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
    }

    response = requests.get(url, headers=headers)
    html_content = response.text
    soup = BeautifulSoup(html_content, 'html.parser')

    dom = etree.HTML(str(soup))
    noticia = dom.xpath('//*[@id="imagenet-content"]/div[2]/div[1]/div/div/div[1]/div[1]/p')[0].text
    titulo = dom.xpath('//*[@id="imagenet-content"]/div[2]/div[1]/div/h2')[0].text
    return noticia, url, titulo

noticias = []
with ThreadPoolExecutor(max_workers=8) as executor:
    for result, url, titulo in executor.map(obter_noticias, links):
        padrao_data = r'\b\d{1,2}/\d{2}/\d{4}\b'
        try:
            data_noticia = re.findall(padrao_data, result)[0]
        except:
            data_noticia = None
        noticias.append({'data': data_noticia, 'titulo': titulo, 'url': url, 'noticia': result})

                

df = pd.DataFrame(noticias)

df.to_csv('tste.csv')